# Using the Custom Hardware Image Generator

This notebook demonstrates hardware generation of an image stream and displaying it on the HDMI output. Before running it you will need to compile the example overlay with the frame generator in Verilog. Two files are generated:
- a `.bit` file, containing the FPGA configuration bitstream
- a `.hwh` (hardware handoff) file, containing information about the module heirarchy and memory map

Copy both files to the user home directory on the Pynq operating system. Then load them (loading the `.bit` also reads the `.hwh` of the same name):

In [1]:
from pynq import Overlay
from pynq.lib.video import *

overlay = Overlay("/home/xilinx/omega_thread.bit")

The overlay object is automatically configured with a Python attribute structure that allows you to access the hardware as Python objects. Appropriate driver classes are instantiated where a hardware block is recognised by the Pynq library. Here we get a convenient handle to the VDMA connected to the image generator:

In [2]:
imgen_vdma = overlay.video.axi_vdma_0.readchannel
videoMode = common.VideoMode(1280, 720, 24) #changed
imgen_vdma.mode = videoMode
imgen_vdma.start()
hdmi_out = overlay.video.hdmi_out
hdmi_out._vdma = overlay.video.axi_vdma #Use the correct VDMA!
hdmi_out.configure(videoMode)
hdmi_out.start()

In [3]:
import socket

# UDP socket for connection to host laptop
RECV_PORT = 20000
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
udp_socket.bind(('0.0.0.0', RECV_PORT))
print("Socket created...")

Socket created...


In [ ]:
from collections import deque
import struct
import threading
import time


# Stores most recent program received from host
recent_program = deque(maxlen=1)
recent_program.append([0,0])    # Prevents error before queue is filled

# Mutex for writing/reading new program
mutex = threading.Lock()


# Send instructions using CPU-GPU control signals
def send_instructions(instructions):
    print("Instruction packet length is", len(instructions))
    program_number = 0
    # Temporary check to avoid rendering null frames
    if (len(instructions) == 2):
        return
    for instr in instructions:
        # First instruction in program
        if (instr & 0xff000000 == 0xa0000000):
            start_instr = instr | (program_number << 24)
            print(hex(start_instr))
            overlay.video.axi_gpio_0.channel1.write(start_instr, 0xffffffff)
            time.sleep(0.1)
        else:
#             print("0xa8000000")
            print(hex(instr))
            overlay.video.axi_gpio_0.channel1.write(0xa8000000, 0xffffffff)
            time.sleep(0.1)
            overlay.video.axi_gpio_0.channel1.write(instr, 0xffffffff)
            time.sleep(0.1)
            
        # exit instruction always at end of program
        if (instr == 0xe0001c00):
#             print("0xb0000000")
            overlay.video.axi_gpio_0.channel1.write(0xb0000000, 0xffffffff)
            time.sleep(0.1)
            program_number += 1
            # Wait for program to finish
            while (overlay.video.axi_gpio_1.channel1.read() != 0b00):
                print("Waiting for program to finish")
                time.sleep(0.001)
                        
    print("Sent instructions to GPU")


# Grabs frame from VDMA and sends it to HDMI
def send_frame():
    frame = imgen_vdma.readframe()
    time.sleep(0.1)
    hdmi_out.writeframe(frame)
    print("Rendered frame")

#     overlay.video.axi_gpio_0.channel1.write(instructions[0], 0xffffffff)
#     while (overlay.video.axi_gpio_1.channel1.read() != 0b10):
#         time.sleep(1)  

#     for i in range(1, len(instructions)):
#         overlay.video.axi_gpio_0.channel1.write(0xa8000000, 0xffffffff)
#         while (overlay.video.axi_gpio_1.channel1.read() != 0b11):
#             time.sleep(1)

#         overlay.video.axi_gpio_0.channel1.write(instructions[i], 0xffffffff)
#         while (overlay.video.axi_gpio_1.channel1.read() != 0b10):
#             time.sleep(1)        

#     overlay.video.axi_gpio_0.channel1.write(0xb0000000, 0xffffffff)
#     while (overlay.video.axi_gpio_1.channel1.read() != 0b01):
#         time.sleep(1)      


# def send_programs(program_list):
#     for program in program_list:
#         send_instructions(program)
#         while (overlay.video.axi_gpio_1.channel1.read() != 0b00):
#             time.sleep(0.001) 


# Receive packets from UDP
# One packet consists of all the assembly programs to render one frame
def udp_recv():
    while True:
        data, addr = udp_socket.recvfrom(RECV_PORT)
        print("Received program over UDP")
        print("Size of UDP packet is", len(data))
        instrs = []
        for i in range(len(data)//4):
            val = struct.unpack("<L", data[4*i:4*(i+1)])[0]
            instrs.append(val)        
        recent_program.append(instrs)


# Just prints the data it receives for now
def send_to_gpu():
    while True:
        # Do not allow new program to be written while copy is happening
        with mutex:
            print("Mutex obtained")
            recent_instructions = recent_program[0].copy()
            print("Mutex lost")
        send_instructions(recent_instructions)
        send_frame()

            
# Run both threads in parallel
recv_thread = threading.Thread(target=udp_recv, daemon=True)           
# gpu_thread = threading.Thread(target=send_to_gpu, daemon=True)

recv_thread.start()
# gpu_thread.start()

while (len(recent_program[0]) == 2):
    time.sleep(0.1)

for i in range(5):
    send_instructions(recent_program[0])

while True:
    send_to_gpu()
#     pass

Received program over UDP
Size of UDP packet is 3008
Instruction packet length is 752
0xa0000200
0x2001a824
0x18084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80

0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xe0001c00
Sent instructions to GPU
Instruction packet length is 752
0xa0000200
0x2001a824
0x18084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480

0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xe0001c00
Sent instructions to GPU
Instruction packet length is 752
0xa0000200
0x2001a824
0x18084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084

0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xe0001c00
Sent instructions to GPU
Instruction packet length is 752
0xa0000200
0x2001a824
0x18084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480

0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xe0001c00
Sent instructions to GPU
Instruction packet length is 752
0xa0000200
0x2001a824
0x18084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480

0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xc0000000
0xe0001c00
Sent instructions to GPU
Mutex obtained
Mutex lost
Instruction packet length is 752
0xa0000200
0x2001a824
0x18084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x80000480
0x20040084
0x8000

Next, the image generator DMA is configured and started:

In [3]:
program0 = [
    0xa0008000,
    0x20000004,
    0x2001a825,
    0x000180a5,
    0x800204a0,
    0xe0001c00
]

program1 = [
    0xa0008000,
    0x2001a825,
    0x000180a5,
    0x20ff9ca4,
    0x60004084,
    0x4452fe06,
    0x60030484,
    0x439c7e46,
    0x60030884,
    0x4c147c06,
    0x60030084,
    0x2004aca6,
    0x210004c6,
    0x00030406,
    0x600040c6,
    0x4452fe07,
    0x600384c6,
    0x439c7e47,
    0x600388c6,
    0x4c147c07,
    0x600380c6,
    0x60030086,
    0x20ff9ca4,
    0x60004084,
    0x4452fe07,
    0x60038484,
    0x439c7e47,
    0x60038884,
    0x4c147c07,
    0x60038084,
    0x2004aca7,
    0x210004e7,
    0x00038407,
    0x600040e7,
    0x4452fe08,
    0x600404e7,
    0x439c7e48,
    0x600408e7,
    0x4c147c08,
    0x600400e7,
    0x44147c08,
    0x4c147c09,
    0x60047080,
    0x70027120,
    0x54147c08,
    0x5c147c09,
    0x700470e0,
    0x7003f120,
    0x54147c08,
    0x5c147c09,
    0x700470c0,
    0x70037120,
    0x53f07c0a,
    0x70050888,
    0x50007c0a,
    0x700508e9,
    0x70048108,
    0x50007c0a,
    0x700508c9,
    0x70048108,
    0x50007c0b,
    0x7005888a,
    0x53f07c0b,
    0x700588e9,
    0x7004814a,
    0x50007c0b,
    0x700588c9,
    0x7004814a,
    0x58007c05,
    0x7002888b,
    0x50007c05,
    0x700288e9,
    0x7004816b,
    0x53f07c05,
    0x700288c9,
    0x7004816b,
    0x20010006,
    0x30000006,
    0x20000165,
    0x43f07c04,
    0x60021104,
    0x60001947,
    0x43a1ff69,
    0x6004f0e0,
    0x58007c07,
    0x70038884,
    0x53a1ff67,
    0x70038084,
    0x70027160,
    0x5c147c05,
    0x43f07c09,
    0x60049149,
    0x60001907,
    0x43a1ff64,
    0x600270e0,
    0x50007c07,
    0x70038929,
    0x53a1ff67,
    0x70038129,
    0x7004f160,
    0x5c147c05,
    0x60001907,
    0x43a1ff69,
    0x6004f0e0,
    0x70001947,
    0x53a1ff69,
    0x7004f0e0,
    0x5c147c05,
    0x200000ab,
    0x2000f0c0,
    0x5c147c05,
    0x7002856b,
    0x53b9fcc5,
    0x7002896b,
    0x50007c05,
    0x7002816b,
    0x5c147c04,
    0x70020508,
    0x54427cc4,
    0x70020908,
    0x5452fe04,
    0x70020108,
    0x5c147c04,
    0x7002054a,
    0x54427cc4,
    0x7002094a,
    0x5452fe04,
    0x7002014a,
    0x70004508,
    0x7000454a,
    0x3100054a,
    0x1005040a,
    0x3004a945,
    0x100400a5,
    0x70004d6b,
    0x90058ca0,
    0xe0001c00
]

program2 = [
    0xa0004100,
    0x20000004,
    0xc0000080,
    0xe0001c00
]

program3 = [
    0xa0002000,
    0x20000004,
    0xc0000080,
    0xc0000080,
    0xc0000080,
    0x20079c26,
    0x2001a8c6,
    0x000180c6,
    0x20022c25,
    0x2004a8a5,
    0x000300a5,
    0x800000a7,
    0x20010008,
    0x30000008,
    0x20000009,
    0x0004f4e0,
    0x300084aa,
    0x90000147,
    0x1004f4e0,
    0x3100054a,
    0x90000147,
    0x1004f4e0,
    0x3000814a,
    0x90000147,
    0x2000f100,
    0xc00000e0,
    0x204000a5,
    0x800000a7,
    0x2001000a,
    0x3000000a,
    0x20000009,
    0x0004f4e0,
    0x300084a8,
    0x90000107,
    0x1004f4e0,
    0x31000508,
    0x90000107,
    0x1004f4e0,
    0x30008108,
    0x90000107,
    0x2000f140,
    0xc00000e0,
    0x204000a5,
    0x800000a7,
    0x20010008,
    0x30000008,
    0x20000009,
    0x0004f4e0,
    0x300084aa,
    0x90000147,
    0x1004f4e0,
    0x3100054a,
    0x90000147,
    0x1004f4e0,
    0x3000814a,
    0x90000147,
    0x2000f100,
    0xc00000e0,
    0x204000a5,
    0x800000a7,
    0x2001000a,
    0x3000000a,
    0x20000009,
    0x0004f4e0,
    0x300084a8,
    0x90000107,
    0x1004f4e0,
    0x31000508,
    0x90000107,
    0x1004f4e0,
    0x30008108,
    0x90000107,
    0x2000f140,
    0xc00000e0,
    0xc0000080,
    0xc0000080,
    0xc0000080,
    0xe0001c00
]

program4 = [
    0xa0004100,
    0x20000004,
    0xc0000080,
    0xe0001c00
]

program_list = [program0, program1, program2, program3, program4]

default_instructions = []
for program in program_list:
    for instruction in program:
        default_instructions.append(instruction)



The frame can be displayed in the notebook using PiL:

In [5]:
import PIL.Image

image = PIL.Image.fromarray(frame)
image

NameError: name 'frame' is not defined

The HDMI output is handled with a wrapper object that configures both the VDMA and the HDMI output generator.

A bit of hacking is needed here: the image generator and its VDMA are in the same part of the design hierarchy (`video`) as the HDMI interface. The HDMI output wrapper `overlay.video.hdmi_out` picks up the image generator VDMA, not the one connected to HDMI, so that attribute needs to be changed to point to the correct VDMA.

After that, the HDMI output is configured and started:

Now our frame can be sent to the HDMI output. At this stage ownership of the frame is transferred to the hardware, so it's no longer accessible to Python code. Connect a display to the HDMI port and you should see it.

Finally, stop the VDMAs so that the system is in a stable state at the end of the notebook

In [8]:
imgen_vdma.stop()
hdmi_out.close()

Todo:
- [ ] Copy multiple frames and find frame rate
- [ ] Link input and output for continuous video output without software control
- [ ] Change image generator settings with MMIO